<a href="https://colab.research.google.com/github/Abdullah-NI/ML-project/blob/main/06%20deploying%20ml%20model%20as%20public%20api/41_Deploying_ML_model_as_public_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pickle5
  Running setup.py clean for pickle5
Failed to build pickle5
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pickle5)


In [16]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [17]:
app=FastAPI()

In [18]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [19]:
class model_input(BaseModel):

    Pregnancies : int
    Glucose : int
    BloodPressure : int
    SkinThickness :int
    Insulin : int
    BMI : float
    DiabetesPedigreeFunction : float
    Age : int

In [20]:
#loading the saved model
diabetes_model=pickle.load(open('/content/drive/MyDrive/AllDataset/diabetes_model.sav','rb'))

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

In [21]:
@app.post('/diabetes_prediction')
def diabetes_pred(input_parameters : model_input):
    input_data=input_parameters.json()
    input_dictionary=json.loads(input_data)

    preg= input_dictionary['Pregnancies']
    glu= input_dictionary['Glucose']
    bp= input_dictionary['BloodPressure']
    skin= input_dictionary['SkinThickness']
    insulin= input_dictionary['Insulin']
    bmi= input_dictionary['BMI']
    dpf= input_dictionary['DiabetesPedigreeFunction']
    age= input_dictionary['Age']


    input_list=[preg,glu,bp,skin,insulin,bmi,dpf,age]

    prediction=diabetes_model.predict([input_list])

    if prediction[0]==0:
        return 'The person is not Diabetic'
    else:
        return 'The person is Diabetic'

In [22]:
ngrok.set_auth_token("2slJa5rurmrQbKFauj8QGYtxFIN_7RJyMg3B2ELSnaxAunDgA")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://b710-35-226-148-208.ngrok-free.app


INFO:     Started server process [389]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     136.232.148.166:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK


<ipython-input-21-2970504c3aca>:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  input_data=input_parameters.json()
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [389]
